In [392]:
# Imports

import gspread as gs
import pandas as pd
import plotly.graph_objects as go

In [393]:
# Match Results Google Sheet
sheet_id = "1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8"
sheet_name = "Match_Results_2"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
results_df = pd.read_csv(url).fillna('')
results_df.head(50)

,Timestamp,Match Week,Match Winner,Match Loser,Games Won By Loser,player1,player2,player1score,player2score
0,,,Missy Kayko,Bob Sauchelli,,Missy Kayko,Bob Sauchelli,1,0
1,,,Adam Fratino,Pat Murphy,,Adam Fratino,Pat Murphy,1,0
2,,,Mike Dunlap,Damir Uzunic,,Mike Dunlap,Damir Uzunic,1,0
3,,,Eric Papa,Kofi Wilson,,Eric Papa,Kofi Wilson,1,0
4,,,Kurowska,Amelia Burger,,Kurowska,Amelia Burger,1,0
5,,,Eric Papa,Kurowska,,Eric Papa,Kurowska,1,0
6,,,Missy Kayko,Paul Assad,,Missy Kayko,Paul Assad,1,0
7,,,Mike Dunlap,Eric Papa,,Mike Dunlap,Eric Papa,1,0
8,,,Missy Kayko,Mike Dunlap,,Missy Kayko,Mike Dunlap,1,0
9,,,Justin Goodfellow,Demelo,,Justin Goodfellow,Demelo,1,0


In [394]:
# Creating Won_Loss Results Table
winners = dict(enumerate(results_df['Match Winner']))
losers = dict(enumerate(results_df['Match Loser']))
winners.update(losers)
players = winners
wins = results_df['player1'].value_counts()
losses = results_df['player2'].value_counts()
w_l = pd.concat([wins,losses],axis=1).fillna(0).astype(int).round().rename(columns={'player1':'Wins','player2':'Losses'})
w_l.loc[:,'Matches']=w_l.sum(numeric_only=True, axis=1)
w_l['Win%'] = w_l['Wins']/w_l['Matches']
w_l['Player']=w_l.index
column_names = ['Player',"Matches", "Wins", "Losses", "Win%"]
w_l['Win%'] = w_l['Win%'].round(3)
w_l = w_l.reindex(columns=column_names)
w_l = w_l.reset_index(drop=True)
w_l.head(10)

,Player,Matches,Wins,Losses,Win%
0,Missy Kayko,7,6,1,0.857
1,Mike Dunlap,7,6,1,0.857
2,Ben Brown,5,5,0,1.000
3,Eric Papa,5,4,1,0.800
4,Jeff Ziev,6,4,2,0.667
5,Ryan Leggette,4,3,1,0.750
6,Nelson Bakerman,3,2,1,0.667
7,Paul Assad,3,2,1,0.667
8,Ian Ainley,4,2,2,0.500
9,Jeff Weber,3,2,1,0.667


In [395]:
# Rankings
rankings_df = pd.read_csv('Rankings.csv')
rankings_df.drop_duplicates(subset=['Player'],inplace=True)
rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.loc[:,'ELO']=1600
initial_ranking = 1600
k_factor = 32
rankings_df.head(20)

,Player,ELO
Player,,
Ben Brown,Ben Brown,1600
Mike Dunlap,Mike Dunlap,1600
Missy Kayko,Missy Kayko,1600
Eric Papa,Eric Papa,1600
Jeff Ziev,Jeff Ziev,1600
Ryan Leggette,Ryan Leggette,1600
Chris Widgren,Chris Widgren,1600
Nelson Bakerman,Nelson Bakerman,1600
Paul Assad,Paul Assad,1600


In [396]:
# ELO Formula
def elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor):
    R1 = 10**(p1_ranking/400)
    R2 = 10**(p2_ranking/400)
    exp_1 = R1/(R1+R2)
    exp_2 = R2/(R1+R2)
    ELO_1 = p1_ranking+k_factor*(p1_score-exp_1)
    ELO_2 = p2_ranking+k_factor*(p2_score-exp_2)
    return ELO_1, ELO_2

In [397]:
#this loop reads the scores in order starting from the top of the matches
#every time you run this loop it will read all of the scores again
for idx in results_df.index:
    player_1 = results_df.player1[idx]
    p1_score = results_df.player1score[idx]
    player_2 = results_df.player2[idx]
    p2_score = results_df.player2score[idx]
    current_rankings_list = list(set(rankings_df.index))
    if player_1 in current_rankings_list:
        p1_ranking = rankings_df.at[player_1,'ELO']
    else:
        p1_ranking = initial_ranking
    if player_2 in current_rankings_list:
        p2_ranking = rankings_df.at[player_2,'ELO']
    else:
        p2_ranking = initial_ranking
    
    ELO_1, ELO_2 = elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor)
    
    if player_1 in current_rankings_list:
        rankings_df.at[player_1,'ELO'] = ELO_1
    else:
        new_ELO_dict = {'Player':player_1,'ELO':ELO_1}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
    if player_2 in current_rankings_list:
        rankings_df.at[player_2,'ELO'] = ELO_2
    else:
        new_ELO_dict = {'Player':player_2,'ELO':ELO_2}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.reset_index(drop=True,inplace=True)
rankings_df.sort_values(by=['ELO'],inplace=True,ascending=False,ignore_index=True)
rankings_df.index+=1
rankings_df.to_csv('Rankings.csv',index=False)

In [398]:
pd.set_option('display.max_rows', None)
rankings_df

,Player,ELO
1,Ben Brown,1678
2,Mike Dunlap,1671
3,Missy Kayko,1670
4,Eric Papa,1645
5,Jeff Ziev,1630
6,Ryan Leggette,1630
7,Chris Widgren,1616
8,Nelson Bakerman,1616
9,Paul Assad,1616
10,Tom Witteman,1616


In [399]:
# Formatting and Printing Final Table
final_table = w_l.merge(rankings_df, how="left", on="Player")
final_table_print = final_table.drop('Matches', axis=1)
final_table_print['Ranking'] = final_table_print[["ELO","Win%"]].apply(tuple,axis=1)\
             .rank(ascending=False, method="min").astype(int)
final_table_print['Ranking'] = final_table_print['Ranking'].round(3)
final_table_print['ELO'] = final_table_print['ELO'].round(0)
column_names = ['Ranking','Player', "ELO", "Wins", "Losses", "Win%"]
final_table_print = final_table_print.reindex(columns=column_names)
final_table_print = final_table_print.sort_values(by=['Ranking','Win%'],ascending=[True,False])
final_table_print


,Ranking,Player,ELO,Wins,Losses,Win%
2,1,Ben Brown,1678,5,0,1.000
1,2,Mike Dunlap,1671,6,1,0.857
0,3,Missy Kayko,1670,6,1,0.857
3,4,Eric Papa,1645,4,1,0.800
5,5,Ryan Leggette,1630,3,1,0.750
4,6,Jeff Ziev,1630,4,2,0.667
15,7,Chris Widgren,1616,1,0,1.000
17,7,George Brown,1616,1,0,1.000
20,7,Tom Witteman,1616,1,0,1.000
6,10,Nelson Bakerman,1616,2,1,0.667


In [400]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
    header=dict(values=list(column_names),
                fill_color='darkblue',
                font_color='white',
                font_size=16,
                align='center'),
    cells=dict(values=[
                final_table_print.Ranking,
                final_table_print.Player,
                final_table_print.ELO,
                final_table_print.Wins,
                final_table_print.Losses, 
                final_table_print['Win%']
                ],
               fill_color='white',
               font_color='black',
               font_size=14,
               align='center',
               height=30,
               line_color='lightgray',
               format=["1234","","#","#","#",".1%"]
               ),
    columnwidth=20)])

fig.show()
fig.write_html("final_table.html", include_plotlyjs="cdn")

final_table_print.to_csv('Rankings_Table.csv',index=False)
final_table_print.to_excel('Rankings_Table.xlsx',index=False)